# Sprint11 深層学習スクラッチ 畳み込みニューラルネットワーク１
* 畳み込みニューラルネットワーク（CNN） のクラスをスクラッチで作成する
* NumPyなど最低限のライブラリのみを使いアルゴリズムを実装する
* このSprintでは1次元の **畳み込み層** を作成し、畳み込みの基礎を理解することを目指す
* 次のSprintでは2次元畳み込み層とプーリング層を作成することで、一般的に画像に対して利用されるCNNを完成させる
* クラスの構造などは前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にする

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import tensorflow as tf
import time
import matplotlib.pyplot as plt

# １次元畳み込み層とは
* CNNでは画像に対しての2次元畳み込み層が定番だが、ここでは理解しやすくするためにまずは1次元畳み込み層を実装する。
* 1次元畳み込みは実用上は自然言語や波形データなどの **系列データ** で使われることが多い
* 畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまではフレームワークで一般的に用意されている
## データセットの用意
* 検証には引き続きMNISTデータセットを使用する
* 1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力する

In [4]:
# MNISTデータのダウンロード
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 平滑化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
# 型の変換
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# trainデータをtrainとvalに分ける
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print('X_train_shape',X_train.shape)
print('X_val_shape',X_val.shape)
print('y_train_shape',y_train.shape)
print('y_test_shape',y_test.shape)
# 仮データ作成
from sklearn.model_selection import train_test_split
X_mini_train, X_mini_test, y_mini_train, y_mini_test = train_test_split(X_test, y_test, test_size=0.1)

Using TensorFlow backend.


X_train_shape (48000, 784)
X_val_shape (12000, 784)
y_train_shape (48000,)
y_test_shape (10000,)


# im２colによる展開
フィルターを適用する入力データの場所（３次元のブロック）を横方向１列に展開する                   
多くの場合，フィルターの適用領域が重なるため，展開後の要素数は元のブロックの要素数より多くなる（それでもfor文で計算するより高速である）                       
im2colで入力データを展開してしまえば，フィルター（重み）を１列に展開して２つの行列の積をとるだけでよくなる

# 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
* チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成（チャネル=色の数）
* 基本構造は前のSprintで作成した全結合層のFCクラスと同じ
* 違いは，データの形状を保持したまま学習する点
* 重みの初期化に関するクラスは必要に応じて作り変える
* Xavierの初期値などを使う点は全結合層と同様
* ここでは **パディング** は考えず、**ストライド**も1に固定する（パディングとストライドで出力サイズが調整できる）
#### パディング                            
入力データの周囲を０などの固定の値で埋めること．パディングを大きくすると，出力サイズも大きくなる．             
→やる理由：出力サイズの調整．(4,4)のデータに(3,3)フィルターをかけると(2,2)になり，畳み込み演算を繰り返すうちに最終的に出力サイズが１になってしまうのを避けるため
#### ストライド
フィルターを適用する位置の間隔．ストライドを大きくすると出力サイズは小さくなる．
* 複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応させる
#### チャネル
色のことかな？                  
#### フィルター
* ニューラルネットワークの時でいう「重み」
* 入力データとフィルターのチャネル数は同じ値にする
* フィルターサイズは好きな値を設定することができる（それが出力のチャネル数になる）
* ただし，チャネルごとのフィルターサイズはすべて同じにしなければならない．
#### バイアス
* CNNのバイアスは常に一つ
* その一つの値をフィルター適用後のすべての要素に足す

### フォワードプロパゲーション
私的フォワードプロパゲーションの解釈               
入力された特徴量に対して任意のサイズ（n×m）の重み（=フィルター）を任意のストライドでずらしながら積和し，出力を求めるフェーズ
$$
a_i = \sum_{s=0}^{F-1}x(i+s)+b
$$
### 更新式
$$
w'_s=w_s-α\frac{\partial L}{\partial w_s}\\
b'=b-α\frac{\partial L}{\partial b}
$$
### バックプロパゲーション

$$\frac{\partial L}{\partial w_s}=\sum_{i=0}^{N^{out-1}}\frac{\partial L}{\partial a_i}x_{(i+s)}\\
\frac{\partial L}{\partial b}=\sum_{i=0}^{N^{out-1}}\frac{\partial L}{\partial a_i}
$$
### 前の層に流す誤差
$$
\frac{\partial L}{\partial x_j}=\sum_{s=0}^{F-1}\frac{\partial L}{\partial a_{(j-s)}}w_s
$$
$$ただし，j-s<0または，j-s>N_{out}-1のとき，\frac{\partial L}{\partial a_{j-s}}=0です$$                
全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差を全て足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。

# 【問題2】1次元畳み込み後の出力サイズの計算
* 畳み込みを行うと特徴量の数が変化する
* パディングやストライドも含めている
* 式は以下
$$
N_{out}=\frac{N_{in} + 2P - F}{S}+1
$$

# 【問題3】小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認する
## 実装上の工夫
畳み込みを実装する場合は、まずはfor文を重ねていく形で構いません。しかし、できるだけ計算は効率化させたいため、以下の式を一度に計算する方法を考えることにします。
$$a_i=\sum_{s=0}^{F-1}x_{(i+s)}w_s+b$$              
バイアス項は単純な足し算のため、重みの部分を見ます。
$$\sum_{s=0}^{F-1}x_{(i+s)}w_s$$               
これは、xの一部を取り出した配列とwの配列の内積です。具体的な状況を考えると、以下のようなコードで計算できます。この例では流れを分かりやすくするために、各要素同士でアダマール積を計算してから合計を計算しています。これは結果的に内積と同様です。

In [5]:
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])

a = np.empty((2, 3))

indexes0 = np.array([0, 1, 2]).astype(np.int)
indexes1 = np.array([1, 2, 3]).astype(np.int)

a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である

a = a.sum(axis=1)

ndarrayは配列を使ったインデックス指定ができることを利用した方法です。

また、二次元配列を使えば一次元配列から二次元配列が取り出せます。

In [6]:
x = np.array([1, 2, 3, 4])
indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)

print(x[indexes]) # ([[1, 2, 3], [2, 3, 4]])

[[1 2 3]
 [2 3 4]]


このこととブロードキャストなどをうまく組み合わせることで、一度にまとめて計算することも可能です。

畳み込みの計算方法に正解はないので、自分なりに効率化していってください。

### 《参考》

以下のページのInteger array indexingの部分がこの方法についての記述です。             
https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html

In [7]:
# 入力x、重みw、バイアスbを次のようにする
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
#フォワードプロパゲーションをすると出力
#a = np.array([35, 50])
#loss
delta_a = np.array([10, 20])
# バックプロパゲーションをすると次のような値
#delta_b = np.array([30])
#delta_w = np.array([50, 80, 110])
#delta_x = np.array([30, 110, 170, 140])

In [8]:
class SGD:
    """確率的勾配降下法"""
    def __init__(self, lr):
        self.lr = lr
    
    def update(self, layer):
        # 更新式
        layer.W = layer.W - self.lr*layer.delta_W
        layer.B = layer.B- self.lr*layer.delta_B
        

In [9]:
class SimpleConv1d:
    def __init__(self,w,b,pad,stride,optimizer,lr):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = w
        self.B = b
        self.p = pad
        self.s = stride
        self.f_size = w.size
        self.optimizer = optimizer(lr)
        
        
    def n_out_put(self,n):
        n_out = int((n + 2*self.p - self.f_size/self.s) + 1)
        return n_out

    def forward(self,x):
        self.x = x
        n_out = self.n_out_put(x.size)  #出力サイズを計算
        a = np.zeros(n_out)             #出力の初期化
        for i in range(n_out):
            #xのi～フィルタサイズ分までの値にフィルタをかける
            a[i] = x[i:i+self.f_size]@self.W + self.B
        return a
     
    def back(self,da):
        self.delta_W = np.zeros(self.W.size)
        self.delta_B = np.sum(da)
        for i in range(self.W.size):
            self.delta_W[i] = x[i:i + da.size]@da
        
        delta_x = []
        for j in range(len(x)):
            temp = 0
            for s in range(len(self.W)):
                if (j-s >= 0) and (j-s < len(da)):
                    temp += da[j-s]*self.W[s]
            delta_x.append(temp)
            
        self.optimizer.update(self)
        return delta_x
    
    def fit(self,x,da):
        n = x.size
        n_out = self.n_out_put(n)
        
        self.a = self.forward(x)
        self.delta_x = self.back(da)
        

In [10]:
# 初期化，学習，確認
conv1 = SimpleConv1d(w=w,b=b,pad=0,stride=1,optimizer=SGD,lr=0.01)
conv1.fit(x,delta_a)
print('delta_x',conv1.delta_x)
print('delta_w',conv1.delta_W)
print('delta_b',conv1.delta_B)
print('更新後のw',conv1.W)
print('更新後のb',conv1.B)

delta_x [30, 110, 170, 140]
delta_w [ 50.  80. 110.]
delta_b 30
更新後のw [2.5 4.2 5.9]
更新後のb [0.7]


# 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成
チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。                     
入力が2チャンネル、出力が3チャンネルの例です。計算グラフを書いた上で、バックプロパゲーションも手計算で考えてみましょう。計算グラフの中には和と積しか登場しないので、微分を新たに考える必要はありません。

### 《補足》

チャンネル数を加える場合、配列をどういう順番にするかという問題があります。**(バッチサイズ、チャンネル数、特徴量数)**または**(バッチサイズ、特徴量数、チャンネル数)**が一般的で、ライブラリによって順番は異なっています。（切り替えて使用できるものもあります）

今回のスクラッチでは自身の実装上どちらが効率的かを考えて選んでください。上記の例ではバッチサイズは考えておらず、**(チャンネル数、特徴量数)**です。



In [11]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）
#a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）である。
delta_a = np.array([[52,56], [32,35], [9,11]])
#FN,FC,FW = w.shape
C,W = x.shape
stride = 1
pad = 0
#da_N,da_W = delta_a.shape
#da_N,da_W

In [12]:
class OUTPUT:
    """一次減畳み込みの出力サイズ"""
    def __init__(self, n,pad,f_size,strider):
        self.n = n #入力サイズ
        self.p = pad #パディングサイズ
        self.f_size = f_size #フィルタサイズ
        self.s = stride #ストライド
        
    def n_out_put(self):
        n_out = int((self.n + 2*self.p - self.f_size/self.s) + 1)
        return n_out

In [13]:
class Conv1d:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, w, b , optimizer,stride, pad):
        self.optimizer = optimizer
        self.W = w
        self.B = b
        self.s = stride
        self.p = pad
        self.FN, self.FC, self.FW = self.W.shape
    
    def _forward(self, X):
        """
        フォワードプロパゲーション
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.A = X
        feature_num = self.A.shape[1]
        
        a = np.zeros([self.FN, feature_num - 2])
        for output in range(self.FN):
            for j in range(self.FW - 1):
                sig = 0
                for chanel in range(self.FC):
                    for i in range(self.FW):
                        sig += X[chanel, i+j] * self.W[output, chanel, j]
                a[output, j] = sig + self.B[output]
        
        return a

    
    def _back(self, dA):
        """
        バックプロパゲーション
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        output = OUTPUT(self.A.shape[0],self.p,self.FW,self.s)
        self.n_out = output.n_out_put()
        self.delta_B = np.sum(dA, axis=1)
        feature_num = self.A.shape[1]
        
        #delta_Wの計算
        self.delta_W = np.zeros_like(self.W)
        for output in range(self.FN):    
            for chanel in range(self.FC):    
                for i in range(self.FW):
                    for j in range(self.FW -1):
                        self.delta_W[output, chanel, i] += dA[output, j]*self.A[chanel, j+i]
                    
        
        #dZの計算
        dZ = np.zeros_like(self.A)
        for output in range(self.FN):
            for chanel in range(self.FC):
                for j in range(feature_num):
                    sigma=0
                    for s in range(self.FW):
                        if j - s < 0 or j - s > self.n_out -1:
                            pass
                        else:
                            sigma += dA[output,  j-s] * self.W[output, chanel, s]
                            
                    dZ[chanel, j] += sigma
        
        # 更新
        self.optimizer.update(self)
        return dZ

In [14]:
cnn = Conv1d(w, b, SGD(0.01), stride, pad)
print(cnn._forward(x))
cnn._back(delta_a)

[[16. 22.]
 [17. 23.]
 [18. 24.]]


array([[0, 0, 0, 0],
       [0, 0, 0, 0]])

# 検証
# 【問題8】学習と推定
これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えてMNISTを学習・推定し、Accuracyを計算してください。

出力層だけは全結合層をそのまま使ってください。ただし、チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、 **平滑化** を行なってください。

画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。
## 重みの初期化クラス
今回は，活性化関数にReLUを使用するため，Heとする

In [15]:
class HeInitializer:
    """
    ReLUの時の重みの初期値
    """
    def W(self, FN, FC, FW):
        """
        重みの初期化
        Parameters
        ----------
        FN : int アウトプットサイズ
        FC : int チャネル数
        FW : int フィルタサイズ
        """
        self.sigma = np.sqrt(2/FN)
        W = self.sigma * np.random.randn((FN,FC,FW))
        return W
    
    def B(self, FN):
        """
        バイアスの初期化
        Parameters
        ----------
        FN : int アウトプットサイズ
        """
        B = self.sigma * np.random.randn(FN)
        return B

## 最適化手法クラス
今回は確率的勾配降下法とする

In [16]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    
    
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        layer.W = layer.W - self.lr*layer.dW
        layer.B = layer.B - self.lr*layer.dB

## 活性化関数
### ReLU

In [17]:
class ReLU:
    def forward(self,A):
        self.A = A
        Z = np.maximum(0,A)
        return Z 
    
    def backward(self,dZ):
        #np.signでAを-1，0，1に変換
        dA = dZ * (np.maximum(0,np.sign(self.A)))
        return dA

### ソフトマックス

In [18]:
class Softmax:
    """
    ソフトマックス関数
    
    """
    
    def forward(self,A3):
        exp_A = np.exp(A3)
        sum_A = np.sum(exp_A,axis=1).reshape(-1,1)
        Z3 = exp_A/sum_A
        return Z3
    
    def backward(self,Z3,Y):
        batch_n = Y.shape[0]
        # A3の勾配
        dA3 = Z3 - Y
        #print('soft_max_dA3',dA3.shape)
        
        # 交差エントロピー誤差
        loss = -np.sum(Y*np.log(Z3+1e-7)) / batch_n
             
        return dA3,loss

## 出力サイズを求める関数

In [19]:
def output_size(n_in,pad,FW,stride):
    """
    一次元の出力サイズを求める関数
    Parameters
    -----------------
    n_in   : int　特徴量の数
    pad : int パディングの数
    FW  : int フィルタのサイズ
    stride   : int ストライドのサイズ  
    Return
    ------------------
    n_out : 出力の特徴量数
    """
    n_out = int((n_in + 2*pad - FW/stride) + 1)
    return n_out

## 畳み込み層

In [20]:
class Conv1d:
    """
    畳み込み層
    Parameters
    ----------
    optimizer:最適化手法
    initializer:重みとバイアスの初期化方法
    FN : int アウトプットサイズ
    FC : int チャネル数（入力のチャネルと同じ）
    FW : int フィルタサイズ
    """
    def __init__(self, optimizer,initializer, FN, FC, FW, pad, stride):
        self.optimizer = optimizer
        self.FN = FN
        self.FC = FC
        self.FW = FW
        self.pad = pad
        self.s = stride
        self.W = initializer.W(FN, FC, FW)
        self.B = initializer.B(FN)
    
    def _forward(self, X):
        """
        フォワードプロパゲーション
        Parameters
        ----------
        X : 次の形のndarray, shape (input_chanel,feature_num)
        Return
        ----------
        A : 次の形のndarray, shape (FN,OW)
        """
        self.Z = X
        self.feature_num = X.shape[1]
        
        A = np.zeros([self.FN, feature_num - 2])
        
        for output in range(self.FN):
            for j in range(self.FW - 1):
                sig = 0
                for chanel in range(self.FC):
                    for i in range(self.FW):
                        sig += X[chanel, i+j] * self.W[output, chanel, j]
                A[output, j] = sig + self.B[output]
        
        return A

    
    def _back(self, dA):
        """
        バックプロパゲーション
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        n_out = output_size(self.feature_num, self.pad, self.FW, self.s)
        self.delta_B = np.sum(dA, axis=1)
        feature_num = self.A.shape[1]
        
        #delta_Wの計算
        self.dW = np.zeros_like(self.W)
        for output in range(self.FN):    
            for chanel in range(self.FC):    
                for i in range(self.FW):
                    for j in range(self.FW -1):
                        self.dW[output, chanel, i] += dA[output, j]*self.Z[chanel, j+i]
                    
        
        #dZの計算
        dZ = np.zeros_like(self.Z)
        for output in range(self.FN):
            for chanel in range(self.FC):
                for j in range(self.feature_num):
                    sigma=0
                    for s in range(self.FW):
                        if j - s < 0 or j - s > n_out -1:
                            pass
                        else:
                            sigma += dA[output,  j-s] * self.W[output, chanel, s]
                            
                    dZ[chanel, j] += sigma
        
        # 更新
        self.optimizer.update(self)
        return dZ

In [21]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス（ガウス分布とか）
    optimizer : 最適化手法のインスタンス（確率的勾配降下とか）
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = initializer.W(n_nodes1,n_nodes2)
        self.B = initializer.B(n_nodes2)
        self.optimizer = optimizer
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力（最初はX,以降はZ）
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.Z = X
        A = np.dot(self.Z,self.W) + self.B
        return A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ = dA@self.W.T
        self.dW = self.Z.T@dA
        self.dB = np.sum(dA,axis=0)
        
        self.optimizer.update(self)
        return dZ

In [22]:
class Scratch1dCNN:
    def __init__(self, lr, epochs, FN,FW,initializer, optimizer, activation, pad, stride):
        #self.lr = lr
        self.epochs = epochs
        self.initializer = initializer()     #重み・バイアスの初期化方法
        self.optimizer = optimizer(lr)       #最適化手法のインスタンス
        self.activation = activation         # 活性化関数
        self.FN = FN
        self.FW = FW
        self.pad = pad
        self.s = stride
    
    def fit(self,X,y):
        # lossを入れる箱
        self.loss = []
        self.val_loss = []
        self.C, self.feature_num = X.shape[0]
        y_out = np.unique(y).size
        x_out = output_size(self.feature_num,self.pad, self.FW,self.s)
        
        # 正解データをOne-Hot表現に
        from sklearn.preprocessing import OneHotEncoder
        enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
        y_one_hot = enc.fit_transform(y[:, np.newaxis])
        
        # 畳み込み層
        self.conv = Conv1d(self.optimizer, self.initializer, self.FN, self.FC, self.FW, self.pad, self.s)
        #活性化層
        self.activation1 = self.activation()
        #全結合層
        fc = FC(x_out, y_out, self.initializer, self.optimizer)
        self.activation2 = Softmax()
        
        for e in range(self.epochs):
            #lossの初期化
            loss = 0
            # ミニバッチの取得
            #get_mini_batch = GetMiniBatch(X, y_one_hot, batch_size=self.batch_n)
            # ミニバッチ開始
            #for mini_X_train, mini_y_train in get_mini_batch:
            # フォワードプロパゲーション
            A1 = self.conv._forward(X)
            Z1 = self.activation1.forward(A1)
            A2 = self.fc.forward(Z1)
            Z2 = self.activation2.forward(A2)
            #A3 = self.FC3.forward(Z2)
            #Z3 = self.activation3.forward(A3)

            #softmax & 交差エントロピー
            dA2,loss = self.activation2.backward(Z2, mini_y_train)
            loss += loss

            # バックプロパゲーション
            dZ2 = self.fc.backward(dA2)
            dA1 = self.activation2.backward(dZ2)
            dZ0 = self.conv._back(dA1)
                        
            #verboseをTrueにした際は学習過程などを出力する
            if self.verbose is True:
                print('Train Data Loss epoch {0} : {1}'.format(e,self.loss[-1]))
                print('Validation Loss epoch {0} : {1}'.format(e,self.val_loss[-1]))
            else:
                pass


    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
        pred : 次の形のndarray, shape (n_samples, 1)  推定結果
        """
        A1 = self.conv._forward(X)
        Z1 = self.activation1.forward(A1)
        A2 = self.fc.forward(Z1)
        Z2 = self.activation2.forward(A2)
        pred = np.argmax(Z2,axis=1)
        return pred

In [23]:
# initializer=初期化(Simple,He,Xa)，optimizer=最適化手法(SGD or AdaGrad)，activation=活性化関数(Sigmoid,Tanh,ReLU)
cnn = Scratch1dCNN(lr=0.01, epochs=5, 
                    FN=3,FW=3,initializer=HeInitializer, optimizer=SGD, activation=ReLU, pad=0, stride=1)

In [24]:
cnn.fit(X_mini_train,y_mini_train)

TypeError: cannot unpack non-iterable int object

In [17]:
#バックプロパゲーションのdelta_Wを求めるフェーズ
delt_w = np.zeros(FN*FC*FW).reshape(FN,FC,FW)
#delta_aのチャネル数分分けて計算するイメージ
for dac in range(da_N):
    #出来上がるdelta_w分計算するイメージ
    for fw in range(FW):
        #dwのFN番目の全部のチャネルの横方向番目を計算していく
        delt_w[dac,:,fw] = np.sum(delta_a[dac,:]*x[:,fw:fw+da_W],axis=1)
        #delt_w[fn,:,fw] = np.sum(delta_a[fn,:]*x[:,fw:fw+da_W])
delt_w

array([[[164., 272., 380.],
        [272., 380., 488.]],

       [[102., 169., 236.],
        [169., 236., 303.]],

       [[ 31.,  51.,  71.],
        [ 51.,  71.,  91.]]])

# 【問題5】（アドバンス課題）パディングの実装
畳み込み層にパディングの機能を加えてください。1次元配列の場合、前後にn個特徴量を増やせるようにしてください。

最も単純なパディングは全て0で埋める ゼロパディング であり、CNNでは一般的です。他に端の値を繰り返す方法などもあります。

フレームワークによっては、元の入力のサイズを保つようにという指定をすることができます。この機能も持たせておくと便利です。なお、NumPyにはパディングの関数が存在します。

# 【問題6】（アドバンス課題）ミニバッチへの対応
ここまでの課題はバッチサイズ1で良いとしてきました。しかし、実際は全結合層同様にミニバッチ学習が行われます。Conv1dクラスを複数のデータが同時に計算できるように変更してください。

# 【問題7】（アドバンス課題）任意のストライド数
ストライドは1限定の実装をしてきましたが、任意のストライド数に対応できるようにしてください